In [1]:
import os
import glob

import numpy as np
import pandas as pd

# Don't install PyNIO into any environment that also has xarray
import xarray as xr

import metpy.calc as mpcalc
from metpy.units import units

from haversine import inverse_haversine, Unit, Direction

import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib as mpl
from matplotlib.pyplot import cm
from matplotlib.colors import ListedColormap, LinearSegmentedColormap
from matplotlib.collections import LineCollection
import matplotlib.lines as mlines
import matplotlib.colors as mcolors
import matplotlib.patheffects as pe
import cartopy.crs as ccrs
import cartopy.feature as cfeature

#from uviz.utils.tools import find_TC_bbox
#from uviz.datashader_tools.utils import datashader_wrapper
#from uviz.plotting.utils import basin_bboxes, ssh_wsp, ssh_mslp, sshws_color, geog_features

In [2]:
tracks_dir = r"/gpfs/group/cmz5202/default/cnd5285/synth_events"
track_folders = glob.glob(os.path.join(tracks_dir, '*storm*'))
parent_folders = [os.path.join(folder, '28km') for folder in track_folders]
child_folders = [os.path.join(folder, '3km') for folder in track_folders]

In [3]:
# Native parent files
h2pn_files = [glob.glob(os.path.join(f, '*h2*[!nc_remap].nc')) for f in parent_folders]
h3pn_files = [glob.glob(os.path.join(f, '*h3*[!nc_remap].nc')) for f in parent_folders]
h4pn_files = [glob.glob(os.path.join(f, '*h4*[!nc_remap].nc')) for f in parent_folders]

# Regridded parent files
h2pr_files = [glob.glob(os.path.join(f, '*h2*.nc_remap.nc')) for f in parent_folders]
h3pr_files = [glob.glob(os.path.join(f, '*h3*.nc_remap.nc')) for f in parent_folders]
h4pr_files = [glob.glob(os.path.join(f, '*h4*.nc_remap.nc')) for f in parent_folders]

# Parent mesh file
# CAM mesh is the "ext" one, based on original model run. Other options are "ref" or "wat"
p_mesh_EXT = r"/gpfs/group/cmz5202/default/cnd5285/maps_and_grids/ne0np4natlanticext.ne30x4.g_scrip.nc"
p_mesh_REF = r"/gpfs/group/cmz5202/default/cnd5285/maps_and_grids/ne0np4natlanticref.ne30x4.g_scrip.nc"
p_mesh_WAT = r"/gpfs/group/cmz5202/default/cnd5285/maps_and_grids/ne0np4natlanticwat.ne30x4.g_scrip.nc"

# Native child files
h2cn_files = [glob.glob(os.path.join(f, '*h2*[!nc_remap].nc')) for f in child_folders]
h3cn_files = [glob.glob(os.path.join(f, '*h3*[!nc_remap].nc')) for f in child_folders]
h4cn_files = [glob.glob(os.path.join(f, '*h4*[!nc_remap].nc')) for f in child_folders]

# Regridded child files
h2cr_files = [glob.glob(os.path.join(f, '*h2*.nc_remap.nc')) for f in child_folders]
h3cr_files = [glob.glob(os.path.join(f, '*h3*.nc_remap.nc')) for f in child_folders]
h4cr_files = [glob.glob(os.path.join(f, '*h4*.nc_remap.nc')) for f in child_folders]

# Child mesh file
c_mesh = "/gpfs/group/cmz5202/default/cnd5285/MPAS_3km/x20.835586.florida.init.CAM.nc"

In [4]:
# # Reads in tracks, assigns to array of xarray datasets
# h2pn_ds = [xr.open_mfdataset(folder, parallel=True) for folder in h2pn_files]
# h3pn_ds = [xr.open_mfdataset(folder, parallel=True) for folder in h3pn_files]
# h4pn_ds = [xr.open_mfdataset(folder, parallel=True) for folder in h4pn_files]

# h2pr_ds = [xr.open_mfdataset(folder, parallel=True) for folder in h2pr_files]
# h3pr_ds = [xr.open_mfdataset(folder, parallel=True) for folder in h3pr_files]
# h4pr_ds = [xr.open_mfdataset(folder, parallel=True) for folder in h4pr_files]

# h2cn_ds = [xr.open_mfdataset(folder, parallel=True) for folder in h2cn_files]
# h3cn_ds = [xr.open_mfdataset(folder, parallel=True) for folder in h3cn_files]
# h4cn_ds = [xr.open_mfdataset(folder, parallel=True) for folder in h4cn_files]

# h2cr_ds = [xr.open_mfdataset(folder, parallel=True) for folder in h2cr_files]
# h3cr_ds = [xr.open_mfdataset(folder, parallel=True) for folder in h3cr_files]
# h4cr_ds = [xr.open_mfdataset(folder, parallel=True) for folder in h4cr_files]

In [4]:
# Reads in mesh files
#p_mesh_ds_EXT = xr.open_dataset(p_mesh_EXT)
p_mesh_ds_REF = xr.open_dataset(p_mesh_REF)
#p_mesh_ds_WAT = xr.open_dataset(p_mesh_WAT)

c_mesh_ds = xr.open_dataset(c_mesh)

In [5]:
test_mpas_data = xr.open_mfdataset([file for file in h4cn_files[0]], parallel=True) 
#mpas_mesh_ds = Polymesh(c_mesh_ds, test_mpas_data, model='mpas')

In [10]:
?Unit

Init signature: Unit(value, names=None, *, module=None, qualname=None, type=None, start=1)
Docstring:     
Enumeration of supported units.
The full list can be checked by iterating over the class; e.g.
the expression `tuple(Unit)`.
File:           /storage/work/c/cnd5285/mambaforge/envs/uviz/lib/python3.8/site-packages/haversine/haversine.py
Type:           EnumMeta
Subclasses:     


In [15]:
miami_coords = (25.775163, -80.208615)
west_coord = inverse_haversine(miami_coords, 100, Direction.WEST, unit=Unit.KILOMETERS)[1]
east_coord = inverse_haversine(miami_coords, 100, Direction.EAST, unit=Unit.KILOMETERS)[1]
north_coord = inverse_haversine(miami_coords, 100, Direction.NORTH, unit=Unit.KILOMETERS)[0]
south_coord = inverse_haversine(miami_coords, 100, Direction.SOUTH, unit=Unit.KILOMETERS)[0]
miami_bbox = ((west_coord, east_coord), (south_coord, north_coord))
miami_bbox

((-81.20727774271047, -79.2099522572895),
 (24.875842636275458, 26.674483363724537))

In [16]:
test_dict = dict(name='Corrine')
test_dict

{'name': 'Corrine'}

In [17]:
isinstance(test_dict, dict)

True